In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

## Main functions to call
from Models_benchmarks import *
from Preprocess import *
from Performance_Metric import *
import time
import os



In [ ]:
def train_model_loop(trials,BatchSize,X_seq_a,X_cor_a,X_feat_a,y_a,test_size,train_size,num_users,usr_list ):
    gcn_df          = pd.DataFrame(columns = ['Metric', 'Avg', 'std','max','min','trial'])
    conv_df         = pd.DataFrame(columns = ['Metric', 'Avg', 'std','max','min','trial'])
    dump_gcn        = []
    dump_bench      = []
    dump_benchL     = []
    dump_true       = []
    dump_cor        = []
    train_y         = []
    dump_usr        = []


    for i in range(0,trials):
        X_train_feat, X_train_seq, X_train_lap,y_train, scaler_y, X_valid_feat, X_valid_seq,X_valid_lap, y_valid, X_test_feat,X_test_seq, X_test_lap, y_test, X_test_cor,tmp,usr_test = split_scale(X_seq_a,X_cor_a,X_feat_a,y_a,test_size,train_size,num_users,usr_list)  
        pred_gcn,pred_cnv,pred_lstm, history,history_cnv,history_lstm = get_prediction_results(BatchSize,X_train_seq, X_train_lap, X_train_feat, y_train,X_valid_seq, X_valid_lap, X_valid_feat, y_valid,X_test_seq, X_test_lap, X_test_feat,scaler_y,num_users)
        true = scaler_y.inverse_transform(y_test)
        y_train = scaler_y.inverse_transform(y_train)
        dump_gcn.append(pred_gcn)
        dump_bench.append(pred_cnv)
        dump_benchL.append(pred_lstm)
        dump_true.append(true)
        dump_cor.append(X_test_cor)
        train_y.append(y_train)
        dump_usr.append(usr_test)
    return dump_gcn,dump_bench,dump_true,dump_cor,train_y,dump_usr,dump_benchL,history,history_lstm,history_cnv
 
    

In [ ]:

graph_sz    = [10,15]  ## graph size
seq_arr     = [3,5,7]   ## sequence length in terms of number of days
label_index = 2 #  2:happiness, 3:stress
y_id        = "y_".format(label_index)
test_size   = 0.4
train_size  = 0.6
trials      = 11
BatchSize   = 10

gcn_l       = pd.DataFrame(columns=['x', 'f1'])
bench_l     = pd.DataFrame(columns=['x', 'f1'])
dump_gcn    = []
dump_bench  = []
dump_benchL  = []
dump_true   = []
dump_cor    = []
train_y     = []
dump_usr    = []
root            = Path(".") ## adjust path according to where data is placed
root1 = '/Users/maryamkhalid/Research_Emotion_Recognition/Research Spring 2022/GCN_CONV_LSTM_Feb_8'

start = time.time()
for gr in graph_sz:
    num_users = gr
    for seq in seq_arr:
        
        print("graph : ",gr, " seq : ",seq)
        file_name   = "GraphSeq"+str(gr)+"_"+str(seq)+".pkl"
        my_path     = root1 + '/' + 'extracted_data'+ '/'+file_name ## path to where extracted data is stored
        data_file   = open(my_path,'rb')
        unfold      = pickle.load(data_file)
        X_seq_a     = unfold["X_seq_a"]
        X_cor_a     = unfold["X_cor_a"]
        X_feat_a    = unfold["X_feat_a"]
        Y           = unfold["Y"]
        y_a         = Y[y_id]
        usr_list    = unfold["usr_id"]
        
        ## train the models and obtain predictions for test data
        dump_gcn1,dump_bench1,dump_true1,dump_cor1,y_tr,dump_test1,dump_benchL1,_,_,_ = train_model_loop(trials,BatchSize,X_seq_a,X_cor_a,X_feat_a,y_a,test_size,train_size,num_users,usr_list)
        dump_gcn.append(dump_gcn1)
        dump_bench.append(dump_bench1)
        dump_benchL.append(dump_benchL1)
        dump_true.append(dump_true1)
        dump_cor.append(dump_cor1)
        train_y.append(y_tr)
        dump_usr.append(dump_test1)
        
        
        x_metr            = ['degree','close','eigen','between','comm','pagerank']
        col_name          = x_metr 
        y_col             = ['yp','yt','usr']
        col_name.extend(y_col)
        gcn_df            = pd.DataFrame(columns=col_name)
        conv_df           = pd.DataFrame(columns=col_name)
        lstm_df           = pd.DataFrame(columns=col_name)
        outer_loop        = len(dump_gcn)

        for i in range(0,outer_loop):
            for j in range(0,trials):
                main_df  = generate_multi_x_metric_mat(dump_true[i][j],dump_gcn[i][j],dump_cor[i][j],dump_usr[i][j],x_metr)
                gcn_df   = gcn_df.append(main_df,ignore_index = True)  ## Proposed model output metrics
            
        for i in range(0,outer_loop):
            for j in range(0,trials):
                main_dfc  = generate_multi_x_metric_mat(dump_true[i][j],dump_bench[i][j],dump_cor[i][j],dump_usr[i][j],x_metr)
                conv_df    = conv_df.append(main_dfc,ignore_index = True) ## benchmark conv-lstm output metrics
            
        for i in range(0,outer_loop):
            for j in range(0,trials):
                main_dfl  = generate_multi_x_metric_mat(dump_true[i][j],dump_benchL[i][j],dump_cor[i][j],dump_usr[i][j],x_metr)
                lstm_df    = lstm_df.append(main_dfl,ignore_index = True)  ## benchmark lstm output metrics


## save results in the folder Results
        file_name1 = "GCNloop"+str(gr )+"_"+str(seq)+"_"+str(trials)+'_'+str(label_index)+".pkl"
        my_path1   = root / "Results" / file_name1  ## folder to save results df containing centrality metrics, true (yt) and predicted (yp) labels for each user
        data = {"gcn_df":gcn_df,"conv_df":conv_df, "lstm_df":lstm_df }
        with my_path1.open('wb') as fp:
            pickle.dump(data, fp)
   
    